In [1]:
import pandas as pd
#from keras.preprocessing import image
from matplotlib import pyplot
from matplotlib.pyplot import imshow
import split_data as sd
import basic_model as n
import numpy as np
from sklearn.metrics import confusion_matrix
#import Network as net
import vectorised as v
from random import randint
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)
from keras.backend.tensorflow_backend import set_session
set_session(tf.Session(config=config))
import keras
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
config=tf.ConfigProto()
config.gpu_options.allow_growth=True

Using TensorFlow backend.


In [2]:
data=pd.read_csv('/home/snehao/dog_breed/Image_Classification/labels.csv')
print(data.head())
targets_series = pd.Series(data['breed'])
one_hot = pd.get_dummies(targets_series, sparse = True)
one_hot_labels = np.asarray(one_hot)

                                 id             breed
0  000bec180eb18c7604dcecc8fe0dba07       boston_bull
1  001513dfcb2ffafc82cccf4d8bbaba97             dingo
2  001cdf01b096e06d78e9e5112d419397          pekinese
3  00214f311d5d2247d5dfe4fe24b2303d          bluetick
4  0021f9ceb3235effd7fcde7f7538ed62  golden_retriever


In [3]:
x_train,x_test,y_train,y_test=sd.split_data(data,one_hot_labels,.20)
#y_test1=x_test['breed'].values
#print(x_train.nlargest(5,x_train.groupby(['breed']).size()))
x_val,x_test,y_val,y_test=sd.split_data(x_test,y_test,.50)
#print(x_train.groupby(['breed']).size().sort_values())
print(x_val.shape)
print(y_val.shape)
print(x_test.shape)
print(y_test.shape)

(1022, 2)
(1022, 120)
(1023, 2)
(1023, 120)


In [4]:
x=[]
y=[]
#for i in range(100):
for i in range(x_train.shape[0]):
    img = image.load_img('/home/snehao/dog_breed/train/'+x_train.iloc[i][0]+'.jpg',target_size=(224,224,3))
    arr=image.img_to_array(img)
    x.append(arr/255)
    y.append(y_train[i])
x=np.array(x)
y=np.array(y)
print('x_shape',x.shape)
print('y_shape',y.shape)

x_shape (8177, 224, 224, 3)
y_shape (8177, 120)


In [5]:
import cv2
x_val_images=[]
y_val1=[]
#for i in range(100):
for i in range(x_val.shape[0]): 
    
    img = image.load_img('/home/snehao/dog_breed/train/'+x_val.iloc[i][0]+'.jpg',target_size=(224,224,3))
    arr=image.img_to_array(img)
    x_val_images.append(arr/255)
    y_val1.append(y_val[i])
x_val_images=np.array(x_val_images)
y_val1=np.array(y_val1)
print('y_shape',y_val1.shape)
print('x_shape',x_val_images.shape)


y_shape (1022, 120)
x_shape (1022, 224, 224, 3)


In [6]:
from keras.layers import Concatenate,Add,BatchNormalization,Input,Dropout,Dense,Activation,Flatten,MaxPooling2D,Conv2D,ZeroPadding2D
from keras.models import Model,Sequential
from keras.applications.inception_v3 import InceptionV3
base_model = InceptionV3(input_shape=(224,224,3),weights='imagenet', include_top=False,pooling='avg')



In [46]:
callback=keras.callbacks.TensorBoard(log_dir='Graphs1', histogram_freq=0,  
          write_graph=True, write_images=True)

In [38]:
featuresx=base_model.predict(x)

In [8]:
features_val=base_model.predict(x_val_images)


In [9]:
featuresx.shape

(8177, 2048)

In [47]:
from keras import optimizers
input_shape=Input((2048,)) 
x1=Dense(1024, activation='relu')(input_shape) 
x1=Dropout(0.4)(x1) 
x1=Dense(512, activation='relu')(x1) 
#x1=Dropout(0.2)(x1) 
x1=Dense(120, activation='softmax')(x1) 
model=Model(inputs=input_shape,outputs=x1) 
print(model.summary()) 
sgd=optimizers.SGD( momentum=0.0, decay=0.0, nesterov=False) 
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_19 (InputLayer)        (None, 2048)              0         
_________________________________________________________________
dense_51 (Dense)             (None, 1024)              2098176   
_________________________________________________________________
dropout_16 (Dropout)         (None, 1024)              0         
_________________________________________________________________
dense_52 (Dense)             (None, 512)               524800    
_________________________________________________________________
dense_53 (Dense)             (None, 120)               61560     
Total params: 2,684,536
Trainable params: 2,684,536
Non-trainable params: 0
_________________________________________________________________
None


In [48]:
model.fit(featuresx,y,epochs=100,validation_data=(features_val,y_val1),batch_size=150,callbacks=[callback])

Train on 8177 samples, validate on 1022 samples
Epoch 1/100
8177/8177 [==============================] - 1s 83us/step - loss: 4.6642 - acc: 0.0488 - val_loss: 4.1443 - val_acc: 0.2583
Epoch 2/100
8177/8177 [==============================] - 0s 30us/step - loss: 3.8992 - acc: 0.2453 - val_loss: 3.2818 - val_acc: 0.5059
Epoch 3/100
8177/8177 [==============================] - 0s 29us/step - loss: 3.0125 - acc: 0.4625 - val_loss: 2.2788 - val_acc: 0.6184
Epoch 4/100
8177/8177 [==============================] - 0s 29us/step - loss: 2.1606 - acc: 0.5921 - val_loss: 1.5586 - val_acc: 0.6879
Epoch 5/100
8177/8177 [==============================] - 0s 28us/step - loss: 1.5988 - acc: 0.6666 - val_loss: 1.1780 - val_acc: 0.7231
Epoch 6/100
8177/8177 [==============================] - 0s 29us/step - loss: 1.2925 - acc: 0.7015 - val_loss: 1.0011 - val_acc: 0.7329
Epoch 7/100
8177/8177 [==============================] - 0s 29us/step - loss: 1.1100 - acc: 0.7314 - val_loss: 0.9142 - val_acc: 0.7495


8177/8177 [==============================] - 0s 27us/step - loss: 0.2863 - acc: 0.9116 - val_loss: 0.7270 - val_acc: 0.7867
Epoch 61/100
8177/8177 [==============================] - 0s 28us/step - loss: 0.2851 - acc: 0.9082 - val_loss: 0.7233 - val_acc: 0.7818
Epoch 62/100
8177/8177 [==============================] - 0s 28us/step - loss: 0.2840 - acc: 0.9116 - val_loss: 0.7274 - val_acc: 0.7857
Epoch 63/100
8177/8177 [==============================] - 0s 29us/step - loss: 0.2766 - acc: 0.9151 - val_loss: 0.7307 - val_acc: 0.7769
Epoch 64/100
8177/8177 [==============================] - 0s 28us/step - loss: 0.2796 - acc: 0.9156 - val_loss: 0.7313 - val_acc: 0.7828
Epoch 65/100
8177/8177 [==============================] - 0s 27us/step - loss: 0.2716 - acc: 0.9170 - val_loss: 0.7234 - val_acc: 0.7838
Epoch 66/100
8177/8177 [==============================] - 0s 29us/step - loss: 0.2634 - acc: 0.9208 - val_loss: 0.7286 - val_acc: 0.7789
Epoch 67/100
8177/8177 [==============================

In [ ]:
model.save_weights('inception1.h5')


In [ ]:
predics=model.predict(x_test_images)
s=tf.Session()
predics=s.run(tf.argmax(predics,1))
np.savetxt('alexnet_results2.csv',predictions,fmt='%d',delimiter=',')
print(confusion_matrix(y_test1,predics))
